# Managing population growth

In [1]:
import sys
import numpy as np
import pandas as pd
import altair as alt

# mapping to the modules that make the app
sys.path.insert(0, "../..")
sys.path.insert(0, "../../app")

import yaml, ssl, asyncio, pickle, os, ast

ssl._create_default_https_context = ssl._create_unverified_context
asyncio.set_event_loop_policy(asyncio.WindowsSelectorEventLoopPolicy())

import nest_asyncio
# this is required for running in a Jupyter Notebook. 
nest_asyncio.apply()

Much of the Population growth is handled in scripts inside the Azure Function

In [2]:
# mapping to the modules that make the app

sys.path.insert(0, "../..")
sys.path.insert(0, "..")
sys.path.insert(0, "../../functions/popgrowth/")

from app.connectors import cmdb_graph
from app.objects import time
from helpers import test_queries

c = cmdb_graph.CosmosdbClient()



executing local windows deployment


In [3]:
params = yaml.safe_load(open(os.path.join(os.getenv("abspath"),"functions/popgrowth/settings.yml")))
syllables = pickle.load(open(os.path.join(os.getenv("abspath"),"app/creators/specs/syllables.p"), "rb"))
username = 'notebook'

Time is updated by the `time` function. It is always running.

In [4]:
t = time.Time(c)
t.get_current_UTU()
params['currentTime'] = t.params['currentTime']
t

< time at: 2023-07-23T13:22:25.424716+00:00 UTU:28311 >

In [5]:
params

{'pop_health_requirement': 0.6,
 'pop_consumes': 2,
 'starve_damage': 0.05,
 'changing_values': ['conformity',
  'literacy',
  'aggression',
  'constitution',
  'wealth',
  'factionLoyalty'],
 'currentTime': 28311}

For this notebook, setting the `pop_health_requirement` to an arbitrary value just to examine.

In [6]:
params['pop_health_requirement'] = 0.4

## Population Resource Consumption

In [7]:
from tools import consumption

We're going to run a resourse query test to ensure that consumption is happening. To test, update the `objid` with a planet that has consuming pops. 

In [8]:
validate_resources_updated_query = f"""
g.V().has('objid','2972908340606').out('has').valueMap()
"""
print(validate_resources_updated_query)
c.run_query(validate_resources_updated_query)
pre_consumption = pd.DataFrame.from_records(c.clean_nodes(c.res))
pre_consumption


g.V().has('objid','2972908340606').out('has').valueMap()



,volume,name,objid,max_volume,description,replenish_rate,username,objtype,id
0,319,Organic,8679787925290,1013,bilogical material that can be consumed by pops,10.0,BillmanLocal2,resource,8679787925290
1,96,Common Minerals,7928654046397,96,Iron and other common material used in constru...,NaN,BillmanLocal2,resource,7928654046397
2,54,Rare Minerals,3119228965862,54,"lithium, silver and other rare minerals used i...",NaN,BillmanLocal2,resource,3119228965862
3,9626,Water,1324452532458,9626,"H2O ready to be consumed, either frozen or in ...",NaN,BillmanLocal2,resource,1324452532458


In [9]:

all_pops_query = f"""
g.V().has('label','pop').as('pop')
    .local(
        union(
            out('inhabits').as('location'),
            out('isOf').as('species')
            )
            .fold()).as('location','species')
        .path()
        .by(unfold().valueMap().fold())
"""

In [10]:
c.run_query(all_pops_query)
data = c.reduce_res(c.res)
pd.DataFrame(data)

,pop,location,species
0,"{'isIdle': 'false', 'name': 'Obattla Magor', '...","{'name': 'Dorf', 'class': 'terrestrial', 'obji...","{'name': 'Koykudra', 'objid': '4291284873310',..."
1,"{'name': 'Obattla Pe', 'objid': '5100638967625...","{'name': 'Dorf', 'class': 'terrestrial', 'obji...","{'name': 'Koykudra', 'objid': '4291284873310',..."
2,"{'name': 'Obattla Van', 'objid': '180095132276...","{'name': 'Dorf', 'class': 'terrestrial', 'obji...","{'name': 'Koykudra', 'objid': '4291284873310',..."
3,"{'name': 'Obattla Rochi', 'objid': '0692394121...","{'name': 'Dorf', 'class': 'terrestrial', 'obji...","{'name': 'Koykudra', 'objid': '4291284873310',..."
4,"{'name': 'Hassia Sin', 'objid': '0248671359952...","{'name': 'Dorf', 'class': 'terrestrial', 'obji...","{'name': 'Koykudra', 'objid': '4291284873310',..."
...,...,...,...
102,"{'isIdle': 'true', 'health': 0.7, 'wealth': 0....","{'name': 'Taipe', 'class': 'terrestrial', 'obj...","{'name': 'Til', 'objid': '5478087080681', 'con..."
103,"{'isIdle': 'true', 'wealth': 0.5632, 'industry...","{'name': 'Taipe', 'class': 'terrestrial', 'obj...","{'name': 'Til', 'objid': '5478087080681', 'con..."
104,"{'isIdle': 'true', 'wealth': 0.483, 'literacy'...","{'name': 'Taipe', 'class': 'terrestrial', 'obj...","{'name': 'Til', 'objid': '5478087080681', 'con..."
105,"{'isIdle': 'true', 'factionLoyalty': 0.375, 'l...","{'name': 'Taipe', 'class': 'terrestrial', 'obj...","{'name': 'Til', 'objid': '5478087080681', 'con..."


In [11]:
pops_df = pd.DataFrame([d['pop'] for d in data])
pops_df

,isIdle,name,objid,conformity,literacy,aggression,constitution,health,isIn,industry,wealth,factionLoyalty,username,objtype,id
0,false,Obattla Magor,7467985401518,0.421,0.376,0.632,0.338,0.7,8822366435138,0.4850,0.4305,0.505,BillmanLocal2,pop,7467985401518
1,true,Obattla Pe,5100638967625,0.325,0.352,0.606,0.450,0.7,8822366435138,0.5280,0.4400,0.459,BillmanLocal2,pop,5100638967625
2,true,Obattla Van,1800951322768,0.446,0.390,0.549,0.573,0.7,8822366435138,0.5610,0.4755,0.423,BillmanLocal2,pop,1800951322768
3,true,Obattla Rochi,0692394121482,0.452,0.395,0.514,0.546,0.7,8822366435138,0.5300,0.4625,0.354,BillmanLocal2,pop,0692394121482
4,true,Hassia Sin,0248671359952,0.623,0.215,0.704,0.569,0.7,4146242421204,0.6365,0.4258,0.662,BillmanLocal2,pop,0248671359952
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
102,true,Ciki Y,4377394578049,0.366,0.554,0.430,0.456,0.7,0015468195520,0.4430,0.4985,0.412,Billmanserver,pop,4377394578049
103,true,Serzeampro Wil,1772981049949,0.424,0.469,0.647,0.668,0.7,4873232607770,0.6575,0.5632,0.394,Billmanserver,pop,1772981049949
104,true,Ciki Basma,1247897538416,0.510,0.532,0.390,0.478,0.7,0015468195520,0.4340,0.4830,0.602,Billmanserver,pop,1247897538416
105,true,Serzeampro Gradon,8096543483724,0.432,0.502,0.546,0.604,0.7,4873232607770,0.5750,0.5385,0.375,Billmanserver,pop,8096543483724


In [12]:
species_df = pd.DataFrame([d['species'] for d in data])
species_df

,name,objid,consumes,effuses,viral_resilience,habitat_resilience,username,objtype,id
0,Koykudra,4291284873310,[Organic],"[Organic waste, Plastics]",0.7,0.2,BillmanLocal2,species,4291284873310
1,Koykudra,4291284873310,[Organic],"[Organic waste, Plastics]",0.7,0.2,BillmanLocal2,species,4291284873310
2,Koykudra,4291284873310,[Organic],"[Organic waste, Plastics]",0.7,0.2,BillmanLocal2,species,4291284873310
3,Koykudra,4291284873310,[Organic],"[Organic waste, Plastics]",0.7,0.2,BillmanLocal2,species,4291284873310
4,Koykudra,4291284873310,[Organic],"[Organic waste, Plastics]",0.7,0.2,BillmanLocal2,species,4291284873310
...,...,...,...,...,...,...,...,...,...
102,Til,5478087080681,[Organic],"[Organic waste, Plastics]",0.7,0.2,Billmanserver,species,5478087080681
103,Til,5478087080681,[Organic],"[Organic waste, Plastics]",0.7,0.2,Billmanserver,species,5478087080681
104,Til,5478087080681,[Organic],"[Organic waste, Plastics]",0.7,0.2,Billmanserver,species,5478087080681
105,Til,5478087080681,[Organic],"[Organic waste, Plastics]",0.7,0.2,Billmanserver,species,5478087080681


In [13]:
locations_df = pd.DataFrame([d['location'] for d in data])
locations_df

,name,class,objid,radius,mass,orbitsDistance,orbitsId,orbitsName,isSupportsLife,isPopulated,isHomeworld,username,objtype,id
0,Dorf,terrestrial,2972908340606,0.884,0.882,1.057,9570931993836,Lisallos,true,true,true,BillmanLocal2,planet,2972908340606
1,Dorf,terrestrial,2972908340606,0.884,0.882,1.057,9570931993836,Lisallos,true,true,true,BillmanLocal2,planet,2972908340606
2,Dorf,terrestrial,2972908340606,0.884,0.882,1.057,9570931993836,Lisallos,true,true,true,BillmanLocal2,planet,2972908340606
3,Dorf,terrestrial,2972908340606,0.884,0.882,1.057,9570931993836,Lisallos,true,true,true,BillmanLocal2,planet,2972908340606
4,Dorf,terrestrial,2972908340606,0.884,0.882,1.057,9570931993836,Lisallos,true,true,true,BillmanLocal2,planet,2972908340606
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
102,Taipe,terrestrial,8490894854096,0.898,0.113,1.087,1849212388331,Lindraja,true,true,true,Billmanserver,planet,8490894854096
103,Taipe,terrestrial,8490894854096,0.898,0.113,1.087,1849212388331,Lindraja,true,true,true,Billmanserver,planet,8490894854096
104,Taipe,terrestrial,8490894854096,0.898,0.113,1.087,1849212388331,Lindraja,true,true,true,Billmanserver,planet,8490894854096
105,Taipe,terrestrial,8490894854096,0.898,0.113,1.087,1849212388331,Lindraja,true,true,true,Billmanserver,planet,8490894854096


First getting the list of consumption for each pop.


In [14]:
pops_df,species_df,locations_df = consumption.all_pops_consumption(c)


We expand that dataset to break open the list of people in each pop. 

In [15]:
params

{'pop_health_requirement': 0.4,
 'pop_consumes': 2,
 'starve_damage': 0.05,
 'changing_values': ['conformity',
  'literacy',
  'aggression',
  'constitution',
  'wealth',
  'factionLoyalty'],
 'currentTime': 28311}

In [16]:
consumption_df = consumption.get_consumption_df(locations_df,species_df,params)
consumption_df = consumption.expand_consumption_df(consumption_df)
consumption_df

c:\Users\william.harding\Documents\repos\exoplanets\notebooks\People\../../functions/popgrowth\tools\consumption.py:53: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ser.consumes = j
c:\Users\william.harding\Documents\repos\exoplanets\notebooks\People\../../functions/popgrowth\tools\consumption.py:53: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ser.consumes = j
c:\Users\william.harding\Documents\repos\exoplanets\notebooks\People\../../functions/popgrowth\tools\consumption.py:53: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pa

,location_id,consumes,pop,consumption,multi
0,2816844503430,Organic,27,54,True
1,2972908340606,Organic,38,76,True
2,5640544774717,Organic,35,70,True
3,8490894854096,Organic,7,14,True



Then we get a list of the resources available on that location. 

In [17]:
c.run_query(consumption.make_resource_query(consumption_df))
resources = c.res
resources

[{'labels': [['location'], ['resource']],
  'objects': [{'objid': ['2972908340606'], 'name': ['Dorf']},
   {'volume': [319], 'objid': ['8679787925290'], 'name': ['Organic']}]},
 {'labels': [['location'], ['resource']],
  'objects': [{'objid': ['5640544774717'], 'name': ['Luraszi']},
   {'volume': [555], 'objid': ['4182507921316'], 'name': ['Organic']}]},
 {'labels': [['location'], ['resource']],
  'objects': [{'objid': ['2816844503430'], 'name': ['Penbufergui']},
   {'volume': [636], 'objid': ['6331044458818'], 'name': ['Organic']}]},
 {'labels': [['location'], ['resource']],
  'objects': [{'objid': ['8490894854096'], 'name': ['Taipe']},
   {'volume': [1023], 'objid': ['2144566983866'], 'name': ['Organic']}]}]

In [18]:
consumption_df = consumption.tally_consumption(c,consumption_df,resources)
consumption_df

,location_id,consumes,pop,consumption,multi,available,remaining
0,2816844503430,Organic,27,54,True,636.0,582.0
1,2972908340606,Organic,38,76,True,319.0,243.0
2,5640544774717,Organic,35,70,True,555.0,485.0
3,8490894854096,Organic,7,14,True,1023.0,1009.0


Then we update the resources by taking out the resources that exist. You can check that this has happened in the graph by looking at the location ids. This is a verification query that isn't run in the function application.

In [19]:
consumption_df.apply(lambda x: consumption.make_resource_update_query(c,x),axis=1)

0    None
1    None
2    None
3    None
dtype: object

In [20]:
c.run_query(validate_resources_updated_query)
post_consumption = pd.DataFrame.from_records(c.clean_nodes(c.res))

In [21]:
pre_consumption

,volume,name,objid,max_volume,description,replenish_rate,username,objtype,id
0,319,Organic,8679787925290,1013,bilogical material that can be consumed by pops,10.0,BillmanLocal2,resource,8679787925290
1,96,Common Minerals,7928654046397,96,Iron and other common material used in constru...,NaN,BillmanLocal2,resource,7928654046397
2,54,Rare Minerals,3119228965862,54,"lithium, silver and other rare minerals used i...",NaN,BillmanLocal2,resource,3119228965862
3,9626,Water,1324452532458,9626,"H2O ready to be consumed, either frozen or in ...",NaN,BillmanLocal2,resource,1324452532458


In [22]:
post_consumption

,volume,name,objid,max_volume,description,replenish_rate,username,objtype,id
0,243,Organic,8679787925290,1013,bilogical material that can be consumed by pops,10.0,BillmanLocal2,resource,8679787925290
1,96,Common Minerals,7928654046397,96,Iron and other common material used in constru...,NaN,BillmanLocal2,resource,7928654046397
2,54,Rare Minerals,3119228965862,54,"lithium, silver and other rare minerals used i...",NaN,BillmanLocal2,resource,3119228965862
3,9626,Water,1324452532458,9626,"H2O ready to be consumed, either frozen or in ...",NaN,BillmanLocal2,resource,1324452532458


Now for the starving. Populations without enough food will starve (health declines). Populations with < 0 health will die.

In [23]:
consumption_df

,location_id,consumes,pop,consumption,multi,available,remaining
0,2816844503430,Organic,27,54,True,636.0,582.0
1,2972908340606,Organic,38,76,True,319.0,243.0
2,5640544774717,Organic,35,70,True,555.0,485.0
3,8490894854096,Organic,7,14,True,1023.0,1009.0


In [24]:
consumption_df[consumption_df['remaining']<=0].apply(lambda x: consumption.lower_health(c,params,x),axis=1)

0 pops will starve in nan


,location_id,consumes,pop,consumption,multi,available,remaining


Looking more closely at `lower_health`
This only happens on locations that have < 0 resources. But here we're going to do it regardless

In [25]:
x = consumption_df.loc[0]
print(x)

dead_pop_nodes = []
dead_pop_ids = []
death_event_edges = []
query =f"""
g.V().has('objid','{x.location_id}').as('location').in('inhabits')
    .haslabel('pop').as('pop')
    .out('isOfSpecies').as('species')
    .path()
        .by(valueMap('objid','name'))
        .by(valueMap('name','objid','health','username'))
        .by(valueMap('name','objid','consumes'))
"""
c.run_query(query)
out = c.res
out

location_id    2816844503430
consumes             Organic
pop                       27
consumption               54
multi                   True
available              636.0
remaining              582.0
Name: 0, dtype: object


[]

In [26]:
print(f"{len(out)} pops will starve in {x.location_id}")

0 pops will starve in 2816844503430


We are only going to starve the pop if the pop consumes that resource. 

health just goes down. 

Let's pretend that the pop will die. IF they do, it will create a `death event`.

In [27]:
pop = test_queries.get_random_pop(c,'BillmanLocal2')
pop

{'isIdle': 'false',
 'name': 'Obattla Magor',
 'objid': '7467985401518',
 'conformity': 0.421,
 'literacy': 0.376,
 'aggression': 0.632,
 'constitution': 0.338,
 'health': 0.7,
 'isIn': '8822366435138',
 'industry': 0.485,
 'wealth': 0.4305,
 'factionLoyalty': 0.505,
 'username': 'BillmanLocal2',
 'objtype': 'pop',
 'id': '7467985401518'}

In [28]:
# death_event = consumption.death_by_starvation_event(c.clean_node(i['objects'][0]),i['objects'][1],params)
# death_event

We will also link that event to an appropriate `happendAt` so that we can see it when looking at events related to specific things. 

In [29]:
# print(c.create_custom_edge(death_event, c.clean_node(i['objects'][0]), 'happenedAt').strip())